In [1]:
import torch
from torch import nn
from BERT.Transformer import RecommendationTransformer

%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import pandas as pd
def load_data(path):
    """
        Load events from files and convert to dataframe.
    """
    map_lst=[]
    for f in os.listdir(path):
        file_name=os.path.join(path,f)
        if os.path.isfile(file_name):
            for line in open(file_name):
                obj = json.loads(line.strip())
                if not obj is None:
                    map_lst.append(obj)
    return pd.DataFrame(map_lst)

In [3]:
df = load_data("active1000")

df = df.loc[df["url"] != "http://adressa.no"]
df.head()

,eventId,category,activeTime,title,url,userId,publishtime,time,documentId
1,88128149,nyheter|trondheim,NaN,Dette er «Kystad-saken» i korte trekk - og sak...,http://adressa.no/nyheter/trondheim/2017/03/17...,cx:13675964253951224092221:1wgstmuzo9vjl,2017-03-17T17:39:35.000Z,1490050820,ff9704d73150a5f14bbad454f4f6e0f252f49521
7,1470946317,nyheter|okonomi,34.0,Han er verdens tredje yngste dollarmilliardær,http://adressa.no/nyheter/okonomi/2017/03/20/h...,cx:i7m47c8k5538rd2u:sdl67619goo2,2017-03-20T21:22:42.000Z,1490050834,8aa45aba5d8e2880410040c0f3c10c7a1e447475
8,620605796,100sport|idrettspolitikk,NaN,Her er den nye strategien etter VM-nederlaget,http://adressa.no/100sport/idrettspolitikk/her...,cx:1359724185772431277628:1bo1nngcb9hcr,2017-03-20T19:55:59.000Z,1490050836,dff1b689590e3ba0ce21a88d3295af016c3abefd
9,1543770103,pluss|nyheter,9.0,Ottervik mener Olsø burde informert om avtalen...,http://adressa.no/pluss/nyheter/2017/03/20/ott...,cx:13675964253951224092221:1wgstmuzo9vjl,2017-03-20T20:28:02.000Z,1490050840,8dc005abfab091585891b0af8c9d9a627e4a2729
10,806901198,pluss|nyheter,NaN,- I en så tragisk sak er alle tapere,http://adressa.no/pluss/nyheter/2017/03/20/i-e...,cx:y3ojtj1dah373vh26yiostnfa:3bnuogklc05m0,2017-03-20T18:20:54.000Z,1490050845,9924065d1af706392a6d7bdaab34c3ba04259158


In [4]:
unique_titles = df["title"].unique().tolist()
unique_users = df["userId"].unique().tolist()

# make dataframe from uniqie titles with id
title_df = pd.DataFrame({"title": unique_titles})
user_df = pd.DataFrame({"userId": unique_users})
title_df["id"] = title_df.index
user_df["id"] = user_df.index

title_df.head()

,title,id
0,Dette er «Kystad-saken» i korte trekk - og sak...,0
1,Han er verdens tredje yngste dollarmilliardær,1
2,Her er den nye strategien etter VM-nederlaget,2
3,Ottervik mener Olsø burde informert om avtalen...,3
4,- I en så tragisk sak er alle tapere,4


In [5]:
sorted = df.sort_values(by=["time"], inplace=False)

title_list = title_df["title"].tolist()

# for each title in sorted change with index of title_list
"""sorted["title"] = sorted["title"].apply(lambda x: title_list.index(x))

grouped_readings = sorted.groupby("userId").agg(list)

grouped_readings.head()"""

'sorted["title"] = sorted["title"].apply(lambda x: title_list.index(x))\n\ngrouped_readings = sorted.groupby("userId").agg(list)\n\ngrouped_readings.head()'

In [6]:
title_id_mapping = {
    k: i+2 for i, k in enumerate(title_list)
}

user_id_mapping = {
    k: i+2 for i, k in enumerate(unique_users)
}

In [7]:
title_id_mapping

{'Dette er «Kystad-saken» i korte trekk - og sakens konsekvenser': 2,
 'Han er verdens tredje yngste dollarmilliardær': 3,
 'Her er den nye strategien etter VM-nederlaget': 4,
 'Ottervik mener Olsø burde informert om avtalen for to år siden': 5,
 '- I en så tragisk sak er alle tapere': 6,
 'Vintervær igjen midt i uken, betydelig varmere fra helgen': 7,
 'Ukultur i Adressa': 8,
 'Så lønnsomt er det å «Harry-handle» på apotek i Sverige': 9,
 '- Ikke holdbart': 10,
 'Norsk kvinne alvorlig skadd i minibussulykke i Sverige': 11,
 '«Den ene skal også ha vist fram tissefanten sin»': 12,
 'Trondheimsrebusen: Vet du dette om Trondheim?': 13,
 None: 14,
 'Stjernens forvandling: Fra klovn til kapteinstype': 15,
 'Det er ikke rart at unge søker til porno': 16,
 '- Kultur-husene må lage underholdning selv': 17,
 'Anita har gjenbrukt vinduer fra Torget og murstein fra Solsiden': 18,
 '«Ta ut stikk-kontakten til varmeovnene allerede i dag»': 19,
 '- Hele livet er snudd opp ned': 20,
 'Hvis den engels

In [8]:
title_df["id"] = title_df["id"].apply(lambda x: x+2)

In [9]:
title_df.to_csv("title_id_mapping.csv", index=False)
#grouped_readings.to_csv("grouped_readings.csv", index=False)

In [10]:
df["userId_mapped"] = df["userId"].apply(lambda x: user_id_mapping[x])
df["title_mapped"] = df["title"].apply(lambda x: title_id_mapping[x])

In [11]:
df.head()

,eventId,category,activeTime,title,url,userId,publishtime,time,documentId,userId_mapped,title_mapped
1,88128149,nyheter|trondheim,NaN,Dette er «Kystad-saken» i korte trekk - og sak...,http://adressa.no/nyheter/trondheim/2017/03/17...,cx:13675964253951224092221:1wgstmuzo9vjl,2017-03-17T17:39:35.000Z,1490050820,ff9704d73150a5f14bbad454f4f6e0f252f49521,2,2
7,1470946317,nyheter|okonomi,34.0,Han er verdens tredje yngste dollarmilliardær,http://adressa.no/nyheter/okonomi/2017/03/20/h...,cx:i7m47c8k5538rd2u:sdl67619goo2,2017-03-20T21:22:42.000Z,1490050834,8aa45aba5d8e2880410040c0f3c10c7a1e447475,3,3
8,620605796,100sport|idrettspolitikk,NaN,Her er den nye strategien etter VM-nederlaget,http://adressa.no/100sport/idrettspolitikk/her...,cx:1359724185772431277628:1bo1nngcb9hcr,2017-03-20T19:55:59.000Z,1490050836,dff1b689590e3ba0ce21a88d3295af016c3abefd,4,4
9,1543770103,pluss|nyheter,9.0,Ottervik mener Olsø burde informert om avtalen...,http://adressa.no/pluss/nyheter/2017/03/20/ott...,cx:13675964253951224092221:1wgstmuzo9vjl,2017-03-20T20:28:02.000Z,1490050840,8dc005abfab091585891b0af8c9d9a627e4a2729,2,5
10,806901198,pluss|nyheter,NaN,- I en så tragisk sak er alle tapere,http://adressa.no/pluss/nyheter/2017/03/20/i-e...,cx:y3ojtj1dah373vh26yiostnfa:3bnuogklc05m0,2017-03-20T18:20:54.000Z,1490050845,9924065d1af706392a6d7bdaab34c3ba04259158,5,6


In [12]:
# describe userId_mapped and title_mapped
df["userId_mapped"].describe()
df["title_mapped"].describe()


count    1.157137e+06
mean     4.366536e+03
std      5.022278e+03
min      2.000000e+00
25%      1.400000e+01
50%      2.320000e+03
75%      7.809000e+03
max      1.896000e+04
Name: title_mapped, dtype: float64

In [13]:
# number of unique users
len(unique_users)

# number of unique titles
len(unique_titles)

print("Number of unique users: ", len(unique_users))
print("Number of unique titles: ", len(unique_titles))

Number of unique users:  1000
Number of unique titles:  18959


In [14]:
df.dropna(subset=["activeTime"], inplace=True)

In [15]:
df.head()


,eventId,category,activeTime,title,url,userId,publishtime,time,documentId,userId_mapped,title_mapped
7,1470946317,nyheter|okonomi,34.0,Han er verdens tredje yngste dollarmilliardær,http://adressa.no/nyheter/okonomi/2017/03/20/h...,cx:i7m47c8k5538rd2u:sdl67619goo2,2017-03-20T21:22:42.000Z,1490050834,8aa45aba5d8e2880410040c0f3c10c7a1e447475,3,3
9,1543770103,pluss|nyheter,9.0,Ottervik mener Olsø burde informert om avtalen...,http://adressa.no/pluss/nyheter/2017/03/20/ott...,cx:13675964253951224092221:1wgstmuzo9vjl,2017-03-20T20:28:02.000Z,1490050840,8dc005abfab091585891b0af8c9d9a627e4a2729,2,5
20,1934514084,pluss|nyheter,94.0,Ottervik mener Olsø burde informert om avtalen...,http://adressa.no/pluss/nyheter/2017/03/20/ott...,cx:i7m47c8k5538rd2u:sdl67619goo2,2017-03-20T20:28:02.000Z,1490050887,8dc005abfab091585891b0af8c9d9a627e4a2729,3,5
21,1311071552,nyheter|okonomi,110.0,Han er verdens tredje yngste dollarmilliardær,http://adressa.no/nyheter/okonomi/2017/03/20/h...,cx:ikedcb0p8sirljmj:xru0bz72t9fy,2017-03-20T21:22:42.000Z,1490050889,8aa45aba5d8e2880410040c0f3c10c7a1e447475,8,3
22,467513049,None,76.0,Så lønnsomt er det å «Harry-handle» på apotek ...,http://adressa.no/nyheter/okonomi/2017/03/20/m...,cx:13675964253951224092221:1wgstmuzo9vjl,2017-03-20T19:34:33.000Z,1490050889,2c62bb35352f115ef9bca1aed891e476f5e7731f,2,9


In [16]:
from BERT.Transformer import RecommendationTransformer
from BERT.Dataset import BertDataset
from torch.utils.data import DataLoader

In [17]:
len(unique_titles)

18959

In [18]:
model = RecommendationTransformer(
    len(unique_titles) + 2,
    num_pos=128,
)

In [19]:
model

RecommendationTransformer(
  (encoder): Encoder(
    (word_embedding): Embedding(18961, 256, padding_idx=0)
    (positional_encoding): PositionalEncoder()
    (dropout): Dropout(p=0.1, inplace=False)
    (layer_stack): ModuleList(
      (0): EncoderLayer(
        (slf_attn): MultiHeadAttention(
          (to_q): Linear(in_features=256, out_features=1024, bias=False)
          (to_k): Linear(in_features=256, out_features=1024, bias=False)
          (to_v): Linear(in_features=256, out_features=1024, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
          (union): Linear(in_features=1024, out_features=256, bias=False)
          (attn): ScaledDotProductAttention(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (layer_norm): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
        )
        (pos_ffn): PositionWiseFeedForward(
          (ff): Sequential(
            (0): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
            (1): Li

In [20]:
from BERT.Trainer import trainer

trainer(
    group_column="userId_mapped",
    data_column="title_mapped",
    model=model,
    data=df,
    optimizer=torch.optim.Adam,
    train_history=128,
    epochs=10,
)

Epoch 1
loader: <torch.utils.data.dataloader.DataLoader object at 0x7fd0c954f640>


100%|██████████| 32/32 [02:46<00:00,  5.19s/it]


Train loss: 8.598506271839142, Train accuracy: 0.25585365605934013
Epoch 2
loader: <torch.utils.data.dataloader.DataLoader object at 0x7fd0c954f640>


100%|██████████| 32/32 [02:48<00:00,  5.27s/it]


Train loss: 7.403864204883575, Train accuracy: 0.32088457824449107
Epoch 3
loader: <torch.utils.data.dataloader.DataLoader object at 0x7fd0c954f640>


...

Train loss: 6.805986702442169, Train accuracy: 0.32037224813448173
Epoch 9
loader: <torch.utils.data.dataloader.DataLoader object at 0x7fd0c954f640>


100%|██████████| 32/32 [02:30<00:00,  4.69s/it]


Train loss: 6.7381380051374435, Train accuracy: 0.32032801692644974
Epoch 10
loader: <torch.utils.data.dataloader.DataLoader object at 0x7fd0c954f640>


100%|██████████| 32/32 [02:26<00:00,  4.57s/it]


Train loss: 6.624239265918732, Train accuracy: 0.3263190763754171
